# Hopfield Net

In [81]:
import numpy as np

In [82]:
# Implementation for asynchronous Hopfield neural network
# Note: memory capacity ≃ 0.14*nodes
class Hopfield_Neural_Network:
    def __init__(self,nodes,iterations=100,weights=None):
        self.nodes = nodes
        self.iterations = iterations
        try:
            if weights == None:
                self.weights = np.zeros((nodes,nodes))
        except ValueError:
            self.weights = weights
    
    def store(self,input):
        dW = np.outer(input.transpose(),input)
        np.fill_diagonal(dW,0)
        self.weights += dW
        
    def recall(self,input):
        update_sequence = np.random.choice(self.nodes, self.iterations)
        for node in update_sequence:
            input[node] = np.sign(np.inner(input,self.weights[:,node]))
        return input

In [83]:
hnn = Hopfield_Neural_Network(10)

# Data

### Matrices

In [84]:
import re

In [85]:
class matrix_expansion:
    
    def __init__(self,active_motors):
        self.active_motors = active_motors
        active_sensors = np.array(active_motors*2)
        active_sensors[len(active_motors):] += 14
        self.active_sensors = active_sensors
        self.shape = ()
        
    def load_from_file(self,filename):
        f = open(filename,"r")
        matrix = f.read()
        f.close()
        matrix = re.split(",NEW_ROW,",matrix)
        matrix.pop()
        matrix = np.array([np.array(re.split(",", row)).astype(np.float) for row in matrix])
        self.shape = matrix.shape
        return matrix
        
    def reduced_matrix(self,matrix):
        matrix = matrix[:,self.active_sensors][self.active_motors]
        return matrix
    
    def expanded_matrix(self,reduced_matrix):
        matrix = np.zeros(self.shape)
        flat = reduced_matrix.flatten()
        matrix = np.zeros((14,28))
        k = 0
        for i in active_motors:
            for j in active_sensors:
                matrix[i,j] = flat[k]
                k += 1
        return matrix

In [86]:
active_motors = [1,3,4,5,10,12]
expander = matrix_expansion(active_motors)

In [87]:
# Front back
filename = "/home/markus/dep/dep_matrices/front_back.dep"
fb_matrix = expander.load_from_file(filename)
fb_reduced = expander.reduced_matrix(fb_matrix)
#fb_expanded = expander.expanded_matrix(fb_reduced)

In [88]:
# Front side
filename = "/home/markus/dep/dep_matrices/front_side.dep"
fs_matrix = expander.load_from_file(filename)
fs_reduced = expander.reduced_matrix(fs_matrix)
#fs_expanded = expander.expanded_matrix(fs_reduced)

In [89]:
# Side down
filename = "/home/markus/dep/dep_matrices/side_down.dep"
sd_matrix = expander.load_from_file(filename)
sd_reduced = expander.reduced_matrix(sd_matrix)
#sd_expanded = expander.expanded_matrix(sd_reduced)

In [90]:
matrices = {"fb": fb_reduced, "fs": fs_reduced, "sd": sd_reduced, "zero": np.zeros(fb_reduced.shape)}

### Transition points

In [111]:
behaviors = ["fb","fs","sd"]
transitions = {("fb","fs"): [], ("fb","sd"): [], ("fs","fb"): [], ("fs","sd"): [], ("sd","fb"): [], ("sd","fs"): []}
transition_muscle_2 = [[-1.5,-1.5,-1.5,-1.5,-1.5,-1.5],[1,1,1,1,1,1]] # [pos,...],[direction,..] -- direction = 1 -> up, direction = 0 -> down

In [112]:
# need to get positions and velocities for fb, fs and sd at -1.5 going up

In [113]:
import pickle

In [114]:
fb = pickle.load(open("/home/markus/dep/dep_data/bases/fb.pickle","rb"))
fs = pickle.load(open("/home/markus/dep/dep_data/bases/fs.pickle","rb"))
sd = pickle.load(open("/home/markus/dep/dep_data/bases/sd.pickle", "rb"))

bases = {"fb": fb, "fs": fs, "sd": sd}

In [115]:
# obtained from plot, time indices that meat transition_muscle_2 condition
fb_t = 124
fs_t = 126
sd_t = 117

In [116]:
# pos data
pos = {"fb": fb[0][fb_t][active_motors], "fs": fs[0][fs_t][active_motors], "sd": sd[0][sd_t][active_motors], "zero": np.zeros(np.array(active_motors).shape)}
# vel data
vel = {"fb": fb[1][fb_t][active_motors], "fs": fs[1][fs_t][active_motors], "sd": sd[1][sd_t][active_motors], "zero": np.zeros(np.array(active_motors).shape)}

### Brain ID

In [117]:
# "zero = [0.0,0.25]"fb" = [0.25,0.5], "fs" = [0.5, 0.75], "sd" = [0.75,1.00]
brain_id = {"zero": 0.125, "fb": 0.375, "fs": 0.625, "sd": 0.875}

# Encoding

In [118]:
# HTM SDR Scalar Encoder
# Input: Scalar
# Parameters: n - number of units, w - bits used to represent signal (width), b - buckets (i.e. resolution), 
#             min - minimum value of input (inclusive), max - maximum input value (inclusive)
class scalar_sdr:
    
    def __init__(self, b, w, min_, max_, shape=0):
        if type(b) != int or type(w) != int or type(min_) != float or type(max_) != float:
            raise TypeError("b - buckets must be int, w - width must be int, min_ must be float and max_ must be float")
        self.b = b # must be int
        self.w = w # must be int
        self.min = min_ # must be float
        self.max = max_ # must be float
        self.n = b+w-1 # number of units for encoding
        self.ndarray_shape = shape
        
    def encode(self,input_):
        if input_ > self.max or input_ < self.min:
            raise ValueError("Input outside encoder range!")
        if type(input_) != float:
            raise TypeError("Input must be float!")
        output = np.zeros(self.n)-1
        index = int((input_-self.min)/(self.max-self.min)*self.b)
        output[index:index+self.w] = 1
        return output
    
    def encode_ndarray(self,input_):
        if input_.shape != self.ndarray_shape:
            raise ValueError("Input dimensions do not match specified encoder dimensions!")
        output = []
        for i in np.nditer(input_, order='K'):
            output.append(self.encode(float(i)))
        return np.array(output)

    def decode(self,input_):
        if len(input_) != self.n or len(np.nonzero(input_+1)[0]) != self.w:
            raise TypeError("Input does not correspond to encoder encoded data!")
        output = np.nonzero(input_+1)[0][0]/float(self.b)*(self.max-self.min)+self.min
        return output
    
    def decode_ndarray(self,input_):
        if input_.shape != (reduce(lambda x, y: x*y, self.ndarray_shape),self.n): 
            raise ValueError("Input dimensions do not match specified encoder dimensions!")
        input_ = input_.reshape(self.ndarray_shape+(self.n,))
        output = []
        for i in np.ndindex(self.ndarray_shape):
            output.append(self.decode(input_[i]))
        output = np.array(output).reshape(self.ndarray_shape)
        return output
    
    def set_ndarray_shape(self,shape):
        if type(shape) != tuple:
            raise TypeError("Must provide tuple of array dimensions!")
        self.ndarray_shape = shape

In [119]:
matrix_encoder = scalar_sdr(60,21,-0.3,0.3,(6,12))
pos_encoder = scalar_sdr(500,21,-100000.0,100000.0,(6,))
vel_encoder = scalar_sdr(200,41,-70.0,70.0,(6,))
brain_encoder = scalar_sdr(1000,101,0.0,1.0)

# Training data

In [120]:
# So what data do we need? What system output do we want? What are the conditions?
# Overall input/output vectors (encoded): [state_matrix,matrix,motor_pos,motor_vel,brain_sig]
# Note: 
#      state matrix -- a sensory value from the dep node
#      motor_pos and motor_vel -- sensory values from motors
#      brain_sig -- single scalar input from brain
#      matrix -- the only output of the system, sending a DEP matrix to the dep node

In [121]:
# Scenario 1: system stationary i.e. zeros (technically transition from zero)
# Input: [state_matrix = 0, matrix = 0, motor_pos = 0, motor_vel = 0, brain_id = brain_id]
# Desired output: [state_matrix = 0, matrix = matrices[brain_id], motor_pos = 0, motor_vel = 0, brain_sig = brain_id]
# Training data = desired output for each brain_id

In [194]:
stationary_data = {}
for id_ in brain_id:
    stationary_data[id_] = np.array([])
    
    state_matrix = matrix_encoder.encode_ndarray(matrices["zero"]) # Note: this is not changed since it is sensory
    stationary_data[id_] = np.append(stationary_data[id_],state_matrix.flatten())
    
    matrix = matrix_encoder.encode_ndarray(matrices[id_])
    stationary_data[id_] = np.append(stationary_data[id_], matrix.flatten())
    
    motor_pos = pos_encoder.encode_ndarray(pos["zero"])
    stationary_data[id_] = np.append(stationary_data[id_],motor_pos.flatten())
    
    motor_vel = vel_encoder.encode_ndarray(vel["zero"])
    stationary_data[id_] = np.append(stationary_data[id_],motor_vel.flatten())
    
    brain_sig = brain_encoder.encode(brain_id[id_])
    stationary_data[id_] = np.append(stationary_data[id_],brain_sig.flatten())


In [157]:
# Scenario 2: transition from one behavior to another based on the input brain_id at the correct motor_pos and motor_vel
# Input: [state_matrix = state_matrix, matrix = 0, motor_pos = motor_pos, motor_vel = motor_vel, brain_sig = brain_id]

# Desired output: if system state equals the transition point, transition, else don't i.e. requires "positive" and "negative examples"

# if motor_pos ~= pos[brain_id[ and motor_vel ~= vel[brain_id]:
#     state_matrix = state_matrix, matrix = matrix[brain_id], motor_pos = motor_pos, motor_vel = motor_vel, brain_sig = brain_id]
# else:
#    state_matrix = state_matrix, matrix = state_matrix, motor_pos = motor_pos, motor_vel = motor_vel, brain_sig = brain_id]

# Training data: single positive example for each transition pair, lots of negatives examples for different motor_pos and motor_vel

In [196]:
# transition point positive examples
transitions_positive_data = {}
for transition in transitions:
    transitions_positive_data[transition] = []
    
    m0 = matrix_encoder.encode_ndarray(matrices[transition[0]])
    transitions_positive_data[transition] = np.append(transitions_positive_data[transition], m0.flatten())
    
    m1 = matrix_encoder.encode_ndarray(matrices[transition[1]])
    transitions_positive_data[transition] = np.append(transitions_positive_data[transition], m1.flatten())
    
    p = pos_encoder.encode_ndarray(pos[transition[0]])
    transitions_positive_data[transition] = np.append(transitions_positive_data[transition], p.flatten())
    
    v = vel_encoder.encode_ndarray(vel[transition[0]])
    transitions_positive_data[transition] = np.append(transitions_positive_data[transition], v.flatten())
    
    brain_sig = brain_encoder.encode(brain_id[transition[1]])
    transitions_positive_data[transition] = np.append(transitions_positive_data[transition], brain_sig.flatten())

In [159]:
# transition point negative examples

In [160]:
# setup negative motor_pos and motor_vel data

neg_pos = {"fb": [], "fs": [], "sd": []}
neg_vel = {"fb": [], "fs": [], "sd": []}

steps = [0, 12, 24, 36, 48, 60, 72, 84, 96, 108]

for key in bases:
    for i in steps:
        neg_pos[key].append(bases[key][0][i][active_motors])
        neg_vel[key].append(bases[key][1][i][active_motors])

In [174]:
# obtain negative examples
transitions_negative_data = {}
for transition in transitions:
    transitions_negative_data[transition] = []
    for i in range(len(neg_pos[transition[0]])):
        temp = np.array([])
        
        m0 = matrix_encoder.encode_ndarray(matrices[transition[0]])
        temp = np.append(temp, m0.flatten())

        m1 = m0
        temp = np.append(temp, m1.flatten())

        p = pos_encoder.encode_ndarray(neg_pos[transition[0]][i])
        temp = np.append(temp, p.flatten())

        v = vel_encoder.encode_ndarray(neg_vel[transition[0]][i])
        temp = np.append(temp, v.flatten())

        brain_sig = brain_encoder.encode(brain_id[transition[1]])
        temp = np.append(temp, brain_sig.flatten())
        
        transitions_negative_data[transition].append(temp)

In [189]:
# Scenario 3: keep situation the same, active high brain signal (technically transitioning from self)
maintain = {}
for id_ in matrices:
    if id_ != "zero":
        maintain[id_] = []
        for i in range(0,len(bases[id_][0]),12):
            temp = np.array([])
            m0 = matrix_encoder.encode_ndarray(matrices[id_])
            temp = np.append(temp, m0.flatten())

            m1 = m0
            temp = np.append(temp, m1.flatten())

            p = pos_encoder.encode_ndarray(bases[key][0][i][active_motors])
            temp = np.append(temp, p.flatten())

            v = vel_encoder.encode_ndarray(bases[key][1][i][active_motors])
            temp = np.append(temp, v.flatten())

            braing_sig = brain_encoder.encode(brain_id[id_])
            temp = np.append(temp, braing_sig.flatten())
            maintain[id_].append(temp)

# Train Hopfield

In [168]:
hnn = Hopfield_Neural_Network(17180,200)

In [200]:
# stationary data
for item in stationary_data:
    hnn.store(stationary_data[item])

In [201]:
# positive transitions data
for item in transitions_positive_data:
    hnn.store(transitions_positive_data[item])

In [202]:
# negative transitions
for key in transitions_negative_data:
    for item in transitions_negative_data[key]:
        hnn.store(item)

In [203]:
# maintain status
for key in maintain:
    for item in maintain[key]:
        hnn.store(item)

# Testing